In [1]:
import os
import pandas as pd
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [2]:
df = pd.read_csv("./tweet_emotions.csv")

llm= ChatOpenAI(temperature=0.9)
df.head()

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_14404\338911499.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm= ChatOpenAI(temperature=0.9)


,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
prompt_template = ChatPromptTemplate.from_template(
    "What the best name to describe someone \
        that is feeling this way {feeling}?"
)

In [5]:
chain = LLMChain(llm=llm, prompt=prompt_template)

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_12992\1546045208.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [8]:
feeling = df.content[:3]

In [9]:
chain.run(feeling)

'The best name to describe someone feeling this way would be "sad" or "depressed."'

#### SimpleSequentialChain
Works best when you expect 1 input and 1 output


In [10]:
from langchain.chains import SimpleSequentialChain

second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description to help someone  \
    feeling this way:{feelings_description}"
) 
chain_two = LLMChain(llm=llm, prompt= second_prompt)

In [11]:
overall_simple_chain = SimpleSequentialChain(chains=[chain,chain_two],
                                            verbose=True
                                             )

In [10]:
overall_simple_chain.run(feeling)



> Entering new SimpleSequentialChain chain...
"Anxious and Anticipatory"
Feeling anxious and anticipatory means you are on edge, constantly worrying about the future. Take deep breaths and stay present.

> Finished chain.


'Feeling anxious and anticipatory means you are on edge, constantly worrying about the future. Take deep breaths and stay present.'

Sequential Chain with morethan one input

In [3]:
from langchain.chains import SequentialChain

In [18]:

chain_one = LLMChain(llm=llm, prompt=prompt_template, 
                     output_key="feeling_description"
                    )


In [19]:
prompt_template_2 = ChatPromptTemplate.from_template(
    "Can you summarize the following feelings in 1 sentence:"
    "\n \n {feeling_description}"
)
chain_two = LLMChain(llm=llm, prompt=prompt_template_2, output_key="feeling_summary")

In [ ]:
prompt_template_3 = ChatPromptTemplate.from_template(
    "What the name of this feeling:\n\n{feeling_summary}"
)
chain_three = LLMChain(llm=llm, prompt=prompt_template_3, output_key="feeling_name")

In [21]:

prompt_template_4 = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary of the feeling:"
    "\n\n Summary: {feeling_name}\n\nLanguage: {feeling_name}"
)

chain_four = LLMChain(llm=llm, prompt=prompt_template_4,
                      output_key="followup_message"
                     )


In [24]:
sequential_chain = SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four],
    input_variables = ["feeling"],
    output_variables = ["feeling_description","feeling_summary","followup_message" ],
    verbose=True
)


In [26]:
sequential_chain(feeling)

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_12992\782359686.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sequential_chain(feeling)




> Entering new SequentialChain chain...

> Finished chain.


{'feeling': 0    @tiffanylue i know  i was listenin to bad habi...
 1    Layin n bed with a headache  ughhhh...waitin o...
 2                  Funeral ceremony...gloomy friday...
 Name: content, dtype: object,
 'feeling_description': 'The best name to describe someone feeling this way would be "sad" or "gloomy."',
 'feeling_summary': 'Feeling downcast and disheartened.',
 'followup_message': "I'm sorry to hear that you're feeling dejected. It's completely normal to have moments of sadness or disappointment, but remember that these feelings are temporary and you will get through this. Allow yourself to feel your emotions and then try to focus on the positive aspects of your life. Remember that you are not alone and there are people who care about you. If you ever need someone to talk to, I'm here to listen. Stay strong."}

### Router Chain
RouterChain is a specialized type of chain used to route data or inputs to different chains based on some condition. It's useful when you need to handle multiple possible workflows in a single chain, where each workflow (chain) is triggered based on specific inputs or conditions.

In [9]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [10]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [11]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [12]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [13]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [14]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [15]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [16]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_14404\3038952769.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [18]:
chain.run("Whos is Rhodrick Mhone from malawi?")



> Entering new MultiPromptChain chain...


History: {'input': 'Who is Rhodrick Mhone from Malawi?'}
> Finished chain.


"I am sorry, but I do not have any information on a person named Rhodrick Mhone from Malawi. It is possible that he may be a private individual or a lesser-known figure in Malawi's history. If you have more information or context about Rhodrick Mhone, I may be able to provide more insight."